# Setup

In [4]:
!git clone https://github.com/onlookertozip/SamsungElectronics_Gumi.git

fatal: destination path 'SamsungElectronics_Gumi' already exists and is not an empty directory.


# Part1: Langchain 기초


## 1. 환경 구성

### 1) 라이브러리 설치

In [5]:
!pip install -q langchain langchain-openai tiktoken

### 2) OpenAI 인증키 설정

In [6]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2. LLM Chain

### 1) Prompt + LLM


In [7]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 실행
result = llm.invoke("지구의 자전 주기는?")

In [8]:
result.content

'지구의 자전 주기는 약 24시간입니다. 이것은 하루가 24시간으로 나누어지는 이유이기도 합니다. 지구는 자전하면서 동쪽으로 회전하고 있으며, 이 동쪽 회전으로 인해 낮과 밤이 생기게 됩니다.'

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 이것은 지구가 자신의 축 주위를 한 바퀴 회전하는 데 걸리는 시간을 의미합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 30, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0451a139-776e-4265-94ac-e09e5d3235b5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 53, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간입니다.'

### 2) Multiple Chains

In [12]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The English translation of "미래" is "future".'

In [13]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'미래 (future)는 아직 일어나지 않은 것들에 대한 시간적 개념이다. 이것은 미래에 발생할 것으로 예상되거나 계획된 사건이나 상황을 가리킨다.'

## 3. Prompt
  

### 1) PromptTemplate

In [14]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [15]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [16]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

'Hello, my name is Hong Gil-dong and I am 30 years old.\n\nI cannot call my father father.'

### 2) ChatPromptTemplate

In [18]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [19]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 139,820km로 태양계에서 가장 큰 행성이며, 질량도 가장 큽니다.'

### 3) Message

In [20]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)


messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [21]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 가스 행성으로, 대략 지름이 지구의 11배에 달하며, 태양계 내에서 가장 많은 질량을 가지고 있습니다.'

## 5. Model Parameter


### 1) 모델 클래스 유형

#### LLM

In [22]:
from langchain_openai import OpenAI

llm = OpenAI()

llm.invoke("한국의 대표적인 관광지 3군데를 추천해주세요.")

'\n\n1. 경복궁\n2. 남산 타워\n3. 부산 해운대 해수욕장'

#### ChatModel

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | chat
chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요."})

AIMessage(content='안녕하세요! 한국의 대표적인 관광지 3군데를 추천해 드리겠습니다.\n\n1. 경복궁: 서울에 위치한 경복궁은 조선 시대 왕궁으로서 국내 최대 규모의 궁궐로 유명합니다. 아름다운 전통 건축물과 정원이 있는 곳으로 한국의 역사와 문화를 경험할 수 있는 곳입니다.\n\n2. 부산 해운대해수욕장: 부산의 대표적인 해변으로 유명한 해수욕장으로 깨끗한 백사장과 아름다운 일몰을 즐길 수 있는 곳입니다. 또한 근처에는 다양한 음식점과 상점들이 있어 관광객들에게 인기가 많습니다.\n\n3. 경주: 경주는 한국의 역사와 문화가 깃든 도시로서 세계문화유산으로 등재된 석굴암이 있으며, 첨성대, 안압지, 불국사 등 다양한 관광지가 있습니다. 또한 경주는 아름다운 자연 풍경으로 유명하며, 자전거 타기나 트레킹을 즐기기에도 좋은 곳입니다.\n\n이렇게 한국의 대표적인 관광지 3군데를 추천해 드렸습니다. 부디 즐거운 여행 되시길 바랍니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 439, 'prompt_tokens': 59, 'total_tokens': 498, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1b8b8ac9-1e02-41e1-a03f-4128adc2bad4-0', usage_metadata={

### 2) 모델 파라미터 설정


#### 모델에 직접 파라미터를 전달 (모델 생성 시점)

In [24]:
from langchain_openai import ChatOpenAI

# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 100,          # 생성할 최대 토큰 수
}

kwargs = {
    "frequency_penalty": 0.5,   # 이미 등장한 단어의 재등장 확률
    "presence_penalty": 0.5,    # 새로운 단어의 도입을 장려
    "stop": ["\n"]              # 정지 시퀀스 설정

}

# 모델 인스턴스를 생성할 때 설정
model = ChatOpenAI(model="gpt-3.5-turbo-0125", **params, model_kwargs = kwargs)


# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

# 전체 응답 출력
print(response)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'stop', 'frequency_penalty', 'presence_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


content='태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 142,984km로 태양계에서 가장 크고 무겁습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 29, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a4832fc4-68ea-4014-a513-835f6deb39fd-0' usage_metadata={'input_tokens': 29, 'output_tokens': 52, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


#### 모델에 직접 파라미터를 전달 (모델 호출 시점)

In [25]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 10,          # 생성할 최대 토큰 수
}

# 모델 인스턴스를 호출할 때 전달
response = model.invoke(input=question, **params)

# 문자열 출력
print(response.content)

태양계에서 가장 큰


#### 모델에 추가적인 파라미터를 전달
- bind 메서드를 사용

In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

# # binding 이전 출력
print(before_answer)

# 모델 호출 시 추가적인 인수를 전달하기 위해 bind 메서드 사용 (응답의 최대 길이를 10 토큰으로 제한)
chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

# binding 이후 출력
print(after_answer)

content='태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 139,820km로 태양계에서 가장 큰 행성이며, 대기의 99%는 수소와 헬륨으로 이루어져 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 58, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-e64a1d72-664c-4f9a-b02e-f21678d506ae-0' usage_metadata={'input_tokens': 58, 'output_tokens': 76, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='태양계에서 가장 큰' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 58, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens'

## 6. Output Parsers


### 1) CSV Parser

In [27]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [28]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']

### 2) JSON Parser

In [29]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [31]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [32]:
chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'Bibimbap is a Korean mixed rice dish that is served as a bowl of warm white rice topped with namul (sautéed and seasoned vegetables), gochujang (chili pepper paste), soy sauce, or doenjang (a fermented soybean paste). A raw or fried egg and sliced meat (usually beef) are common additions. The ingredients are stirred together before eating. To cook Bibimbap'}

# Part2: RAG (Retrieval-Augmented Generation)
- 모델의 학습 데이터에 포함되지 않은 데이터를 사용 (환각 방지)
- **외부 데이터**를 검색(retrieval)한 후, 생성(generation) 단계에서 LLM에 전달
- 모델은 주어진 컨텍스트나 질문에 더 적합하고 풍부한 정보를 반영한 답변을 생성
- 논문: https://arxiv.org/abs/2005.11401

## 0. 환경 구성

### 1) 라이브러리 설치

In [33]:
!pip install -q langchain langchain-openai langchain_community tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.8 MB/s eta 0:00:00


In [34]:
import langchain

langchain.__version__

'0.3.7'

### 2) OpenAI 인증키 설정

In [35]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 1. RAG 파이프라인 개요
- Load Data - Text Split - Indexing - Retrieval - Generation



##### Step 1: Load Data

In [36]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
13141
 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만 읽고 나가버리니까요. 각 정책 

##### Step 2: Split texts

In [37]:
# Text Split (Documents -> small chunks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

18
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

In [38]:
# page_content 속성
splits[10].page_content

'제안과 채택\n\xa0백:아님 §\xa0관료주의  문서를 참고하십시오. 단축백:제안\n제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.\n\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.'

In [39]:
# metadata 속성
splits[10].metadata

{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

##### Step 3: Indexing

In [40]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
print(len(docs))
print(docs[0].page_content)

4
격하
특정 정책이나 지침이 편집 관행이나 공동체 규범이 바뀌며 쓸모없어질 수 있고, 다른 문서가 개선되어 내용이 중복될 수 있으며, 불필요한 내용이 증식할 수도 있습니다. 이 경우 편집자들은 정책을 지침으로 격하하거나, 정책 또는 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안할 수 있습니다. 
격하 과정은 채택 과정과 비슷합니다. 일반적으로 토론 문서 내 논의가 시작되고 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단해야 합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.
소수의 공동체 인원만 지지하는 수필, 정보문 및 기타 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동합니다. 이러한 논의는 일반적으로 해당 문서의 토론란에서 이루어지며, 간혹 위키백과:의견 요청을 통해 처리되기도 합니다.

같이 보기
위키백과:위키백과의 정책과 지침 목록
위키백과:의견 요청
수필

위키백과:제품, 절차, 정책
위키백과:위키백과 공동체의 기대와 규범
기타 링크


##### Step 4: Retrieval ~ Generation

In [41]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")

'격하 과정은 특정 정책이나 지침이 더 이상 필요하지 않거나 개선이 필요한 경우, 해당 정책이나 지침을 수정하거나 중단하는 과정을 말합니다. 이를 위해 편집자들은 해당 정책이나 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 변경하거나, 완전히 폐지할 수 있습니다. 이러한 변경이 이루어지기 위해서는 토론이 시작되고, 공동체의 참여를 요청하는 틀이 붙여진 후 논의가 이루어지며, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되어야 합니다. 폐지된 정책이나 지침은 해당 문서 상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 것임을 알립니다.'

### PDF 파일기반 질의응답 챗봇 (랭체인, 그라디오, ChatGPT)

In [58]:
!pip install openai # openai 라이브러리를 설치합니다.
!pip install langchain # 랭체인 라이브러리를 설치합니다.

In [43]:
!pip install pypdf # pdf 로딩용
!pip install chromadb # 벡터스토어
!pip install tiktoken # 토큰 계산용

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.9 MB/s eta 0:00:00


In [59]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [60]:
loader = PyPDFLoader("SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [61]:
texts

[Document(metadata={'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf', 'page': 0}, page_content='- 1 -\n2023년 가을 학술대회 및 제41회 정기총회 안내\n▢ 행사개요\n  ○ 행사명 : 2023년 가을 학술대회 및 제41회 정기총회\n  ○  기   간  :  2 0 2 3 .  1 0 .  2 5 . ( 수 )  ~  2 7 . ( 금 )  \n  ○ 장  소 : 여수 베네치아 호텔&리조트\n             \n▢ 초록 및 프로시딩 접수   \n  ○  접수기간 : 2023. 8. 21.(월) ~ 9. 15.(금)\n  ○ 접수방법 : 홈페이지 -> 학술대회 (초록제출을 위해서는 연회비납부 및 사전등록 필요)\n  ○ 영문초록양식 : 영문초록. 저자명 풀 네임 사용\n  ○ 발표분야 : 특별세션 구성하면서 추가될 수 있음.\n-태양 및 우주환경 (태양, 자기권, 전리층, 고층대기, COSMIC RAY 등 및 관련 관측 장비)\n-우주응용 (측지, 마이크로중력 환경 실험, 원격탐사 및 관련 탑재체)\n-우주기술 (위성운영, 위성체) \n-우주천문 (우주천문 탑재체, 천문 관측기기, 천문학 일반, 고천문, 천문대운영, 소프트웨어) \n-우주감시 (광학 및 레이다 관측, 우주물체궤도 분석, 우주위험 분석) \n-달과 우주탐사: 과학과 기술 그리고 정책 (태양계 탐사, 달탐사)\n-초소형 위성\n-안보우주\n-지상 및 우주 인프라 운영기술\n-특별세션 : Open New Horizon with L4 mission\n▢ 발표자  \n   ○ 논문 제출자는 발표자로 간주하며 제1저자이어야하며 정회원이고 연회비를 완납하여야 함.\n   ○ 초록 마감 후에는 발표형식(구두, 포스터) 조정이 어려우므로 접수시 정확히 선택.\n   ○ 2편 발표자는 구두발표 2편 or 구두발표 1편 + 포스터발표 1편 (포스터 발표만 2편 불가) \n   ○ 발표자료

In [62]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(texts, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True)

In [64]:
query = "음료 지원은?"
result = chain(query)
print(result)

{'question': '음료 지원은?', 'answer': '음료 지원은 없습니다.\n', 'sources': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회 세부안내.pdf', 'source_documents': [Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 042-865-3392)\n  ○ 신청서 : 학회 홈페이지 자료실\n  ○ 인건비 : 300,000원 (숙박,

In [65]:
result['answer']

'음료 지원은 없습니다.\n'

In [66]:
result['sources']

'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회 세부안내.pdf'

In [67]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_template="""Use the following pieces of context to answer the users question shortly.
Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}

You MUST answer in Korean and in Markdown format:"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": prompt}

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)  # Modify model_name if you have access to GPT-4

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [69]:
query = "음료 지원은?"
result = chain(query)
print(result)

{'question': '음료 지원은?', 'answer': '**답변:**\n\n음료에 대한 정보는 제공되지 않았습니다. 따라서 음료에 대한 지원 여부에 대해서는 알 수 없습니다.\n\n**', 'sources': '** SamsungElectronics_Gumi/datasets/2023년 가을 학술대회 세부안내.pdf', 'source_documents': [Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 

In [70]:
result['answer']

'**답변:**\n\n음료에 대한 정보는 제공되지 않았습니다. 따라서 음료에 대한 지원 여부에 대해서는 알 수 없습니다.\n\n**'

In [71]:
result['source_documents']

[Document(metadata={'page': 1, 'source': 'SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf'}, page_content='- 2 -\n  ○ 등록비\n     \n구분 정회원 군경/학부생/\n 전일제 대학원생 비회원\n사전등록비 270,000원 150,000원 320,000\n현장등록비 300,000원 180,000원 320,000\n  ※ 연구소, 또는 기타 기관에서 등록비 지원을 받는 대학원생은 정회원 등록비.(학생 등록비 아님)\n  ○ 식사 : 온라인 신청기간 10월 4일 ~ 13일까지\n    \n구분 조식, 중식(1회 기준) 만찬\n식사비 5,000원 신청자에 한해 지원\n(일부학회지원)\n만찬은 신청한 회원에 한해 지원되며  그 외 식사는 회당 5,000원(25,000원 상당)\n씩 회원 부담입니다.\n등록시 식사 신청을 하지 않은 분들은 현장에서 일반 식사비를 지불해야 합니다.\n가족식사 신청 만찬 2만원 / 26일 중식 1만원  / 조식뷔폐는 가족식권 판매 안함\n(단, 현금 결제, 카드결제 안됨)\n  ○ 지정숙박 : 개별 신청 (신청서 다운로드)\n      \n25일(수), 26일(목) \n 더블 : 100,000원  / 트윈 : 110,000원  /  페밀리 트윈 : 120,000원\n▢ 행사진행 도우미 모집\n  ○ 모집인원 : 00명 \n  ○ 신청기간 : 2023. 10. 2.(월)까지. 심사 후 통보\n  ○ 신청방법 : 신청서 작성하여 신청 (이메일: ksss@ksss.or.kr, 팩스 042-865-3392)\n  ○ 신청서 : 학회 홈페이지 자료실\n  ○ 인건비 : 300,000원 (숙박, 식사 제공)\n  ○ 참여일 : 2023. 10. 25.(수) - 27.(금) \n▢ 학생등록비 지원(선착순 3명)\n  ○  신청자격 : 발표자(공동발표자) 중 연구과제에서 등록비를 지원받을 수 없는 학부생과 정회  \n     원 중 전일제 석, 박

In [72]:
for doc in result['source_documents']:
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))

내용 : - 2 -   ○ 등록비       구분 정회원 군경/학부생/  전일제 대학원생 비회원 사전등록비 270,000원 150,000원 320,000 현장등록비 300,000원 180,
파일 : SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf
페이지 : 1
내용 : - 2 -   ○ 등록비       구분 정회원 군경/학부생/  전일제 대학원생 비회원 사전등록비 270,000원 150,000원 320,000 현장등록비 300,000원 180,
파일 : SamsungElectronics_Gumi/datasets/2023년 가을 학술대회  세부안내.pdf
페이지 : 1


In [73]:
!pip install gradio # 그라디오 라이브러리를 설치합니다.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [74]:
import gradio as gr

In [75]:
def respond(message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    result = chain(message)

    bot_message = result['answer']

    for i, doc in enumerate(result['source_documents']):
        bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') '

    chat_history.append((message, bot_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history  # 수정된 채팅 기록을 반환합니다.

with gr.Blocks() as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [msg, chatbot], [msg, chatbot])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(debug=True)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://673dfb5c7e75bce8f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://673dfb5c7e75bce8f6.gradio.live
